In [1]:
%load_ext autoreload
%autoreload 2
!sudo rm /tmp/libtpu_lockfile
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
import jax
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
max_seq_len = 64
batch_size = 16

In [3]:
from micrlhf.llama import LlamaTransformer
from micrlhf.caching_llama import FoldedLlamaKVCachingTransformer
from micrlhf.flash import flashify

llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4", load_on_cpu=True,
                                        #  transpose_rotary=False
                                        #  transpose_rotary=True
                                         )
print("Folding and caching...")
llama, cache = FoldedLlamaKVCachingTransformer.from_uncached(llama, max_seq_len, {"batch": batch_size})
# llama = flashify(llama)

Folding and caching...


In [4]:
llama = llama.to_tpu()

In [5]:
import gc
gc.collect();

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
], tokenize=False)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
from micrlhf.sampling import sample
texts, cache = sample((llama, cache), tokenizer, prompt, batch_size=batch_size, max_seq_len=max_seq_len,
       do_sample=True, return_only_completion=True)
texts

  0%|          | 0/69 [00:00<?, ?it/s]

ValueError: Custom node type mismatch: expected type: <class 'micrlhf.caching_llama.LlamaKVCachingInputs'>, value: LlamaKVCachingInputs(tokens=<NamedArray int32(| batch:16, seq:1) (wrapping DynamicJaxprTracer)>, positions=<NamedArray int32(| batch:16, seq:1) (wrapping DynamicJaxprTracer)>, attention_mask=<NamedArray bool(| kv_seq:128, batch:16, seq:1) (wrapping DynamicJaxprTracer)>, sampling_state=LlamaKVCachingState(cache_len=128, batch_axes={'batch': 16}, kv_caches={'WithSideInputsFromInputTuple.body/LlamaTransformer.body/WithSideInputsFromInputTuple.body/Sequential.sublayers[3]/ScanSequential.layer/LlamaBlock.sublayers[1]/Residual.delta/Sequential.sublayers[2]/LlamaKVCachingAttention.kv_cache': (<NamedArray bfloat16(| batch:16, projection:128, layer:80, kv_heads:8, seq:128) (wrapping DynamicJaxprTracer)>, <NamedArray bfloat16(| batch:16, projection:128, layer:80, kv_heads:8, seq:128) (wrapping DynamicJaxprTracer)>)}, cache_end_index=Traced<ShapedArray(int32[], weak_type=True)>with<D

In [16]:
next(iter(cache.kv_caches.values()))

(<NamedArray bfloat16(| batch:16, projection:128, layer:80, kv_heads:8, seq:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:167_772_160 (wrapping jax.Array)>,
 <NamedArray bfloat16(| batch:16, projection:128, layer:80, kv_heads:8, seq:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:167_772_160 (wrapping jax.Array)>)